<a href="https://colab.research.google.com/github/SergioMoraa/Parcial_2_TAM_Solucion/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import h5py
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from streamlit_drawable_canvas import st_canvas
from PIL import Image
import warnings

# Ignorar advertencias para una presentación más limpia
warnings.filterwarnings('ignore')

# --- 1. Configuración de la Página y Estilos ---
st.set_page_config(
    page_title="Dashboard Clasificación de Dígitos",
    page_icon="✍️",
    layout="wide"
)

# CSS para personalizar la apariencia del dashboard
st.markdown("""
    <style>
    .main-header {font-size:2.5rem; color:#0048A0; text-align:center; margin-bottom:1rem; font-weight: bold;}
    .sub-header {font-size:1.8rem; color:#0066CC; margin-top:1rem; margin-bottom:1rem; border-bottom: 2px solid #0066CC;}
    .card {background-color:#f9f9f9; padding:1.5rem; border-radius:10px; box-shadow:0 4px 8px rgba(0,0,0,0.1); margin-bottom:1rem; height: 100%;}
    .stButton>button {
        background-color: #0048A0;
        color: white;
        border-radius: 5px;
        border: 2px solid #0048A0;
        font-weight: bold;
    }
    .stButton>button:hover {
        background-color: #0066CC;
        border-color: #0066CC;
        color: white;
    }
    </style>
    """, unsafe_allow_html=True)

# --- 2. Carga y Cacheo de Datos y Modelos ---

@st.cache_data
def load_data(path='usps.h5'):
    """Carga y cachea los datos del archivo HDF5."""
    try:
        with h5py.File(path, 'r') as hf:
            X_train = hf.get('train').get('data')[:]
            y_train = hf.get('train').get('target')[:]
            X_test = hf.get('test').get('data')[:]
            y_test = hf.get('test').get('target')[:]
        return X_train, y_train, X_test, y_test
    except Exception as e:
        st.error(f"Error al cargar el dataset 'usps.h5': {e}")
        return None, None, None, None

@st.cache_resource
def train_models(X_train, y_train):
    """Entrena y cachea los modelos de ML. Se ejecuta solo una vez."""
    with st.spinner('Entrenando modelos... Esto solo se hace una vez.'):
        # Modelo K-Nearest Neighbors (basado en el notebook)
        knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
        knn.fit(X_train, y_train)

        # Modelo Support Vector Machine (basado en el notebook)
        svm = SVC(kernel='rbf', C=10, gamma=0.1, probability=True)
        svm.fit(X_train, y_train)

    models = {'K-NN': knn, 'SVM': svm}
    return models

# --- Carga Principal ---
X_train, y_train, X_test, y_test = load_data()

# Si los datos no se cargan, detener la app
if X_train is None:
    st.stop()

models = train_models(X_train, y_train)
y_preds = {name: model.predict(X_test) for name, model in models.items()}

# --- 3. UI del Dashboard ---
st.markdown("<h1 class='main-header'>Dashboard de Clasificación de Dígitos Manuscritos (USPS)</h1>", unsafe_allow_html=True)
st.markdown("---")

# Creación de pestañas
tab1, tab2, tab3 = st.tabs(["📊 Exploración de Datos", "📈 Evaluación de Modelos", "✍️ Predicción Interactiva"])

# --- Pestaña 1: Exploración de Datos ---
with tab1:
    st.markdown("<h2 class='sub-header'>Análisis Exploratorio del Dataset</h2>", unsafe_allow_html=True)

    col1, col2 = st.columns([1, 1])

    with col1:
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.subheader("Muestra de Dígitos del Dataset")
        st.write("Visualización de algunos ejemplos de imágenes 16x16 y su etiqueta real.")

        if st.button('Mostrar nuevos ejemplos'):
            indices = np.random.choice(len(X_train), 9)
        else:
            indices = np.arange(9)

        fig, axes = plt.subplots(3, 3, figsize=(6, 6))
        for i, ax in enumerate(axes.flat):
            img_index = indices[i]
            image = X_train[img_index].reshape(16, 16)
            ax.imshow(image, cmap='gray_r')
            ax.set_title(f"Label: {y_train[img_index]}", fontsize=12)
            ax.axis('off')
        plt.tight_layout()
        st.pyplot(fig)
        st.markdown("</div>", unsafe_allow_html=True)

    with col2:
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.subheader("Distribución de Clases")
        st.write("Cantidad de imágenes por cada dígito en el conjunto de entrenamiento.")

        class_counts = pd.Series(y_train).value_counts().sort_index()
        fig_bar = px.bar(
            class_counts,
            x=class_counts.index,
            y=class_counts.values,
            labels={'x': 'Dígito', 'y': 'Cantidad'},
            title='Frecuencia de cada Dígito',
            color_discrete_sequence=px.colors.qualitative.Pastel
        )
        fig_bar.update_layout(xaxis_title="Dígito", yaxis_title="Número de Muestras")
        st.plotly_chart(fig_bar, use_container_width=True)
        st.markdown("</div>", unsafe_allow_html=True)

# --- Pestaña 2: Evaluación de Modelos ---
with tab2:
    st.markdown("<h2 class='sub-header'>Rendimiento de los Modelos de Clasificación</h2>", unsafe_allow_html=True)

    st.markdown("<div class='card'>", unsafe_allow_html=True)
    st.subheader("Matrices de Confusión")
    st.write("Las matrices de confusión muestran los aciertos y errores de cada modelo en el conjunto de prueba. En la diagonal principal se encuentran las predicciones correctas.")

    col1, col2 = st.columns(2)

    for i, (name, y_pred) in enumerate(y_preds.items()):
        cm = confusion_matrix(y_test, y_pred)
        fig = ff.create_annotated_heatmap(
            z=cm,
            x=[str(i) for i in range(10)],
            y=[str(i) for i in range(10)],
            colorscale='Blues',
            showscale=True
        )
        fig.update_layout(title_text=f'<b>{name}</b>', xaxis_title="Predicción", yaxis_title="Valor Real")

        if i % 2 == 0:
            with col1:
                st.plotly_chart(fig, use_container_width=True)
        else:
            with col2:
                st.plotly_chart(fig, use_container_width=True)

    st.markdown("</div>", unsafe_allow_html=True)

    st.markdown("<div class='card'>", unsafe_allow_html=True)
    st.subheader("Métricas de Rendimiento")
    st.write("Tabla comparativa con las métricas clave (Precisión, Recall, F1-Score) para cada modelo.")

    metrics_data = []
    for name, y_pred in y_preds.items():
        report = classification_report(y_test, y_pred, output_dict=True)
        metrics_data.append({
            "Modelo": name,
            "Accuracy": f"{report['accuracy']:.2%}",
            "Precision (Ponderada)": f"{report['weighted avg']['precision']:.2%}",
            "Recall (Ponderado)": f"{report['weighted avg']['recall']:.2%}",
            "F1-Score (Ponderado)": f"{report['weighted avg']['f1-score']:.2%}",
        })

    metrics_df = pd.DataFrame(metrics_data).set_index("Modelo")
    st.dataframe(metrics_df, use_container_width=True)
    st.markdown("</div>", unsafe_allow_html=True)

# --- Pestaña 3: Predicción Interactiva ---
with tab3:
    st.markdown("<h2 class='sub-header'>¡Dibuja y Predice tu Propio Dígito!</h2>", unsafe_allow_html=True)

    col1, col2 = st.columns([1, 1])

    with col1:
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.subheader("Lienzo de Dibujo")
        st.write("Dibuja un dígito del 0 al 9 en el siguiente recuadro. Intenta que sea centrado y grande.")

        # Crear un lienzo para dibujar
        canvas_result = st_canvas(
            fill_color="rgba(0, 0, 0, 0)",  # Fondo transparente
            stroke_width=15,
            stroke_color="#FFFFFF", # Dibujar en blanco sobre fondo negro
            background_color="#000000",
            update_streamlit=True,
            height=280,
            width=280,
            drawing_mode="freedraw",
            key="canvas",
        )
        st.markdown("</div>", unsafe_allow_html=True)

    with col2:
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.subheader("Resultado de la Predicción")

        if st.button('🧠 Predecir Dígito'):
            if canvas_result.image_data is not None:
                # Preprocesar la imagen dibujada
                img = Image.fromarray(canvas_result.image_data.astype('uint8'))
                img_gray = img.convert('L') # Convertir a escala de grises
                img_resized = img_gray.resize((16, 16), Image.Resampling.LANCZOS) # Redimensionar a 16x16

                # Convertir a formato numérico y escalar a [-1, 1]
                # Los datos originales están en [-1, 1]. El lienzo da [0, 255].
                # Se normaliza de [0, 255] a [0, 2] y luego se resta 1 para obtener [-1, 1]
                img_array = np.array(img_resized)
                processed_array = (img_array / 255.0) * 2 - 1

                # Aplanar para el modelo
                final_array = processed_array.flatten().reshape(1, -1)

                with st.spinner('Realizando predicción...'):
                    # Realizar predicciones
                    pred_knn_proba = models['K-NN'].predict_proba(final_array)
                    pred_svm_proba = models['SVM'].predict_proba(final_array)

                    pred_knn = np.argmax(pred_knn_proba)
                    pred_svm = np.argmax(pred_svm_proba)

                    st.success("¡Predicción completada!")

                    st.metric(label="Predicción K-NN", value=f"{pred_knn}", delta=f"{pred_knn_proba.max():.2%} de confianza")
                    st.metric(label="Predicción SVM", value=f"{pred_svm}", delta=f"{pred_svm_proba.max():.2%} de confianza")

                    # Mostrar la imagen que "vio" el modelo
                    st.write("Así es como el modelo 'vio' tu dibujo (16x16):")
                    fig, ax = plt.subplots(figsize=(2,2))
                    ax.imshow(final_array.reshape(16, 16), cmap='gray_r')
                    ax.axis('off')
                    st.pyplot(fig)
            else:
                st.warning("Por favor, dibuja un dígito primero.")
        else:
            st.info("Dibuja un dígito en el lienzo y presiona el botón 'Predecir Dígito'.")
        st.markdown("</div>", unsafe_allow_html=True)

# --- Pie de Página ---
st.markdown("---")
st.markdown("<div style='text-align: center; margin-top: 20px;'><p>Dashboard creado con Streamlit por un asistente de IA de Google.</p></div>", unsafe_allow_html=True)